In [1]:
import numpy as np
import random
import math
import multiprocess as mp
import talib as ta
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from datetime import datetime,date
from gga import GGA
from island_ga import IslandGGA 

In [2]:
#rename column to remove spaces in column names
data=pd.read_csv('HistoricalQuotes.csv')
aapl =data.rename(columns={' Close/Last': 'close', ' Volume': 'volume',
                           ' Open':'open',' High':'high',' Low':'low'})
                        

#change date string into date format and sort the dataframe in ascending order
aapl[['Date']] = aapl[['Date']].applymap(lambda x: datetime.strptime(x,'%m/%d/%Y').date())
aapl =aapl.sort_values(by='Date')
#change dataframe dtypes to float
cols = [i for i in aapl if i not in ['Date','volume']]
def to_float(x):
    x = x.strip()
    return float(x[1:])

aapl[cols]=aapl[cols].applymap(to_float)

#Create Signals using Talib library
aapl['5EMA'] = ta.SMA(np.array(aapl['close']),5)
aapl['20EMA'] = ta.EMA(np.array(aapl['close']), timeperiod = 20)
aapl['RSI'] = ta.RSI(np.array(aapl['close']), timeperiod = 14)
aapl['WILLR'] = ta.WILLR(np.array(aapl['high']), np.array(aapl['low']), np.array(aapl['close']), timeperiod=14)
aapl['MOM'] = ta.MOM(np.array(aapl['close']), timeperiod=5)
aapl['CCI'] = ta.CCI(np.array(aapl['high']), np.array(aapl['low']), np.array(aapl['close']), timeperiod=14)
aapl['SLOWK'],aapl['SLOWD'] = ta.STOCH(np.array(aapl['high']), np.array(aapl['low']), np.array(aapl['close']), fastk_period=14, slowk_period=3, slowd_period=3)
aapl['MACD'],aapl['MACDSIGNAL'],aapl['MACDHIST'] = ta.MACD(np.array(aapl['close']), fastperiod=12, slowperiod=26, signalperiod=9)
aapl['DMI'] = ta.DX(np.array(aapl['high']), np.array(aapl['low']), np.array(aapl['close']), timeperiod=14)   

In [4]:
#  """Based on parameter setting adopted in Chen et al (2021) """
def generate_candidate_trading_signals(data):
    """Based on parameter setting adopted in Chen et al (2021) """
    conditions ={'TS1':[
                (data['5EMA'] > data['20EMA']),
                (data['5EMA'] < data['20EMA'])],
                 'TS2':[
                (data['RSI'] > 30),
                (data['RSI'] < 70),
                ],
                 'TS3':[
                (data['WILLR'] < 80),
                (data['WILLR'] > 20),
                ],
                 'TS4':[
                (data['MOM'] > 0 ),
                (data['MOM'] <= 0),
                ],
                 'TS5': [
                (data['CCI'] > 100 ),
                (data['CCI'] <= 100),
                ],
                 'TS6': [
                (data['SLOWK'] > data['SLOWD']) & (data['SLOWD'] < 20),
                (data['SLOWK'] < data['SLOWD']) & (data['SLOWD'] > 80)],
                'TS7': [
                (data['MACD'] > 0 ),
                (data['MACD'] <= 0)],
                'TS8': [
                (data['CCI'] > 100 ),
                (data['CCI'] <= -100)]}

    # create a list of the values we want to assign for each condition 1: buy, 0: sell
    values = [1, 0]

    # create a new column and use np.select to assign values to it using our lists as arguments
    for i in conditions:
        data[i] = np.select(conditions[i], values)
    strategies =list(conditions.keys())
    return data,strategies

In [5]:
#split data
train = aapl[(aapl['Date'] <= date(2016,12,31))]
test = aapl[(aapl['Date'] > date(2016,12,31))]
train_data, strategies =generate_candidate_trading_signals(train.copy())
test_data, test_strategies =generate_candidate_trading_signals(test.copy())
num_islands=3
num_iter=20
pSize=20
m_iter=5
N=5
K=3
r_cross=0.8
r_mut= 0.01
r_inv=0.3
n=4
b=4
stop_loss=-0.15
take_profit=0.15
allocated_capital=1000
islandgga = IslandGGA(num_islands,num_iter,train_data,strategies,pSize,m_iter,N,K,r_cross,r_mut,r_inv,n,b,stop_loss,take_profit,allocated_capital)

In [19]:

print(pop[0])

[[1, 1, 1, 0, 1, 0, 1, 0], [array(['TS5', 'TS2', 'TS1'], dtype='<U3'), array(['TS3', 'TS7', 'TS6'], dtype='<U3'), array(['TS8', 'TS4'], dtype='<U3')], [0, 0, 0, 0, 1, 1, 1], 1]


In [22]:
ts= islandgga.strategy_performance(pop[0][0])
islandgga.getProfit(ts,pop[0])

0.0

In [21]:
islandgga.getWeights(pop[0][2])

{0: [], 1: [], 2: [], 3: []}

In [11]:
pop =islandgga.init_population()

ts= islandgga.strategy_performance(pop[0][0])
islandgga.getCorrelation(ts,pop[0])

          TS4       TS7       TS1
TS4  1.000000 -0.168177 -0.005935
TS7 -0.168177  1.000000  0.408830
TS1 -0.005935  0.408830  1.000000
          TS4       TS7       TS5
TS4  1.000000 -0.168177 -0.289161
TS7 -0.168177  1.000000  0.127457
TS5 -0.289161  0.127457  1.000000
          TS4       TS8       TS1
TS4  1.000000 -0.289161 -0.005935
TS8 -0.289161  1.000000  0.135474
TS1 -0.005935  0.135474  1.000000
          TS4       TS8       TS5
TS4  1.000000 -0.289161 -0.289161
TS8 -0.289161  1.000000  1.000000
TS5 -0.289161  1.000000  1.000000
          TS4       TS2       TS1
TS4  1.000000  0.920857 -0.005935
TS2  0.920857  1.000000  0.054163
TS1 -0.005935  0.054163  1.000000
          TS4       TS2       TS5
TS4  1.000000  0.920857 -0.289161
TS2  0.920857  1.000000 -0.377084
TS5 -0.289161 -0.377084  1.000000
          TS6      TS7       TS1
TS6  1.000000 -0.23195 -0.049123
TS7 -0.231950  1.00000  0.408830
TS1 -0.049123  0.40883  1.000000
          TS6       TS7       TS5
TS6  1.000000 -0.2

12.76744590714522

In [18]:
ring = []
master = []
multi = []
for i in range(50):
    islandgga.evolve_island_ring()
    ring_p4mc = islandgga.evaluate_best_profit(test_data)
    ring.append(islandgga.globalBest)
    islandgga.evolve_master_slave()
    master.append(islandgga.globalBest)
    islandgga.evolve_island_multikuti()
    multi.append(islandgga.globalBest)

In [6]:
islandgga.evolve_island_ring()

In [7]:
islandgga.evaluate_best_profit(test_data)

273.86607460605836